# Molly exercise

This exercise will introduce you more to the Molly.jl package for molecular simulation. Julia 1.9 and Molly 0.18 or later are recommended. Basic familiarity with Julia and molecular dynamics concepts is assumed. You may find the [Molly documentation](https://juliamolsim.github.io/Molly.jl/stable/docs) and the [Julia documentation](https://docs.julialang.org/en/v1) helpful.

First, we import the packages that we need.

In [ ]:
using Molly
using StaticArrays
using Unitful
using Bio3DView
using Plots
using KernelDensity
using Measurements
using Zygote
using LinearAlgebra
using Statistics

For this exercise you will also need the files `dipeptide_nowater.pdb` and `dipeptide_equil.pdb`, which can be found alongside this notebook, in the current directory.

We will be running short simulations of alanine dipeptide, which is the amino acid alanine with an acetyl group on the N-terminus and a N-methylamide group on the C-terminus. The simulations are kept short so they run fairly quickly on any computer. We can get a quick look at `dipeptide_nowater.pdb`, which contains just alanine dipeptide, with Bio3DView.jl.

In [ ]:
viewfile("dipeptide_nowater.pdb", style=Style("stick"))

`dipeptide_equil.pdb` contains alanine dipeptide and water in a periodic simulation box equilibrated to 300 K.

In [ ]:
viewfile("dipeptide_equil.pdb", style=Style("sphere", Dict("scale"=> "0.5")))

### Exercise 1

- Open the above files in PyMOL and familiarise yourself with changing the representation (lines, sticks, spheres etc.) and showing the periodic simulation box (show cell).
- Why is it necessary to use this much water? What would happen if the simulation box tightly surrounded the alanine dipeptide molecule?

---

We are going to simulate the motion of alanine dipeptide with a molecular mechanics force field. First we read in the AMBER99SB-ILDN force field parameters.

In [ ]:
ff_dir = joinpath(dirname(pathof(Molly)), "..", "data", "force_fields")
ff = MolecularForceField(joinpath.(ff_dir, ["ff99SBildn.xml", "tip3p_standard.xml"])...)

Now we set up a `System` using the force field and the structure file. `rename_terminal_res` is a technicality related to residue numbering which is required in this case.

In [ ]:
sys = System("dipeptide_nowater.pdb", ff; rename_terminal_res=false)

We can calculate the potential energy of this conformation of alanine dipeptide (in a vacuum).

In [ ]:
neighbors = find_neighbors(sys)
potential_energy(sys, neighbors)

Notice the type of the potential energy.

In [ ]:
typeof(potential_energy(sys, neighbors))

This is a "unitful" quantity, provided by Unitful.jl. Unitful quantities can be created as follows.

In [ ]:
5.0u"kg"
2.0u"m" + 3.0u"m"
[1.0, 2.0]u"s" .* 2

### Exercise 2

- Add 1 m to 1 nm.
- Try and add 1 m to 1 kg. What happens?

It will also be useful to understand static arrays, which are used to store quantities like 3D coordinates for speed.

In [ ]:
typeof(sys.coords[1])

In [ ]:
SVector(1.0, 2.0, 3.0)

In [ ]:
SVector(1.0, 2.0, 3.0)u"m" .+ SVector(4.0, 5.0, 6.0)u"nm"

### Exercise 3

- Create a `SteepestDescentMinimizer` and run an energy minimization on the system.
- Calculate the new potential energy. How has it changed?

We can apply random velocities drawn from the Maxwell-Boltzmann distribution to the atoms.

In [ ]:
random_velocities!(sys, 300.0u"K")
temperature(sys)

### Exercise 4

- Apply random velocities to the system 1000 times and record the temperature.
- What is the mean and standard deviation of the temperature? Why is the temperature not always the same?

Now we can apply loggers to the system to record properties of interest throughout the simulation.

In [ ]:
sys = System(sys, loggers=(
    energy=TotalEnergyLogger(100),
    temperature=TemperatureLogger(100),
    writer=StructureWriter(100, "trajectory.pdb"),
))

We can simulate for 50 ps with the velocity Verlet integrator.

In [ ]:
simulator_vv = VelocityVerlet(dt=0.0005u"ps")
simulate!(sys, simulator_vv, 100_000)

### Exercise 5

- The stored values of a logger can be accessed with `values(sys.loggers.energy))` or similar. Plot the total energy and the temperature over the simulation. What do you see?
- Open `trajectory.pdb` in PyMOL, align the states and watch the simulation.

We can also simulate in the NVT ensemble using a Langevin integrator.

In [ ]:
simulator_lang = Langevin(dt=0.0005u"ps", temperature=300.0u"K", friction=1.0u"ps^-1")

# Reset the system
sys = System(
    "dipeptide_nowater.pdb",
    ff;
    rename_terminal_res=false,
    loggers=(
        energy=TotalEnergyLogger(100),
        temperature=TemperatureLogger(100),
        writer=StructureWriter(100, "trajectory.pdb"),
    ),
)
random_velocities!(sys, 300.0u"K")

isfile("trajectory.pdb") && rm("trajectory.pdb")
simulate!(sys, simulator_lang, 100_000)

### Exercise 6

- Plot the total energy and the temperature over the simulation. How does this compare the the previous simulation in the NVE ensemble?

The torsion angle between four atoms is an important conformational measure for peptides and proteins. We can calculate the ϕ torsion angle as follows.

In [ ]:
rad2deg(torsion_angle(sys.coords[2], sys.coords[7], sys.coords[9], sys.coords[15], sys.boundary))

To log the ϕ and ψ torsion angles over a simulation, we can use a `GeneralObservableLogger`.

For this simulation we are also going to use the GBNeck2 implicit solvent model to mimic simulation in water.

In [ ]:
function phi_wrapper(sys, args...; kwargs...)
    rad2deg(torsion_angle(sys.coords[2], sys.coords[7], sys.coords[9],
                          sys.coords[15], sys.boundary))
end

function psi_wrapper(sys, args...; kwargs...)
    rad2deg(torsion_angle(sys.coords[7], sys.coords[9], sys.coords[15],
                          sys.coords[17], sys.boundary))
end

sys = System(
    "dipeptide_nowater.pdb",
    ff;
    rename_terminal_res=false,
    loggers=(
        writer=StructureWriter(100, "trajectory.pdb"),
        phi=GeneralObservableLogger(phi_wrapper, Float64, 100),
        psi=GeneralObservableLogger(psi_wrapper, Float64, 100),
    ),
    implicit_solvent="gbn2",
)
random_velocities!(sys, 300.0u"K")

In [ ]:
isfile("trajectory.pdb") && rm("trajectory.pdb")
simulate!(sys, simulator_lang, 200_000) # This will take a little while to run

### Exercise 7

- Plot a scatter plot of the recorded torsion angles with ϕ on the x-axis and ψ on the y-axis. This is a Ramachandran plot.
- Look up the regions of the Ramachandran plot. Which secondary structure is being sampled by the system? What happens if you repeat the simulation?
- Set up and run a similar situation but in vacuum rather than implicit solvent. Compare the Ramachandran plots.

Now we are going to run a simulation in explicit solvent and look at how the O-H distances in water change. We are using a molecular mechanics force field with a harmonic potential for bonds, so we should expect to see that reflected in the dynamics.

For now we just log one of the O-H distances in the system and run a short 100 fs simulation.

In [ ]:
function OH_distance_wrapper(sys, args...; kwargs...)
    ustrip(norm(vector(sys.coords[23], sys.coords[24], sys.boundary)))
end

sys = System(
    "dipeptide_equil.pdb",
    ff;
    rename_terminal_res=false,
    loggers=(
        distance=GeneralObservableLogger(OH_distance_wrapper, Float64, 1),
        writer=StructureWriter(1, "trajectory.pdb"),
    ),
)
random_velocities!(sys, 300.0u"K")

In [ ]:
isfile("trajectory.pdb") && rm("trajectory.pdb")
simulate!(sys, simulator_lang, 200)

### Exercise 8

- Look at the trajectory in PyMOL. What happens when an atom crosses the bounding box?
- Plot the logged O-H distance over the simulation. Explain what you see.
- Estimate roughly from the plot the time per period of oscillation of the bond.
- Using the formula for the frequency of a spring, `f = 1/(2π)×√(k/m)`, estimate the expected time per period based on the utilised `k` of 462750.4 kJ mol^-1 nm^-2 and `m` of 1 g/mol. Does it match the observed value?

Now, we can set up the same system but with deuterated water, where the hydrogens have a mass of 2 amu.

In [ ]:
sys = System(
    "dipeptide_equil.pdb",
    ff;
    rename_terminal_res=false,
    loggers=(
        distance=GeneralObservableLogger(OH_distance_wrapper, Float64, 1),
        writer=StructureWriter(1, "trajectory.pdb"),
    ),
)

deut_atoms = map(sys.atoms, sys.atoms_data) do at, at_data
    if at_data.atom_type == "tip3p-H"
        new_mass = 2.0u"g/mol"
    else
        new_mass = at.mass
    end
    Atom(index=at.index, charge=at.charge, mass=new_mass, σ=at.σ, ϵ=at.ϵ)
end

sys = System(sys; atoms=deut_atoms)
random_velocities!(sys, 300.0u"K")

In [ ]:
isfile("trajectory.pdb") && rm("trajectory.pdb")
simulate!(sys, simulator_lang, 200)

### Exercise 9

- Plot the logged O-H distances and compare them to the previous exercise.
- Again estimate roughly from the plot the time per period of oscillation of the bond. Compare this to the expected value from the formula when `m` is 2 g/mol.

Now we are going to look at how the water is distributed around one of the carbonyl oxygens in alanine dipeptide during the simulation. The atom indices of the water oxygens are:

In [ ]:
const water_O_inds = findall(ad -> ad.atom_type == "tip3p-O", sys.atoms_data)

### Exercise 10

- Write a function `Owat_distance_wrapper` with the same arguments as the wrapper functions above. It should return a `Vector{Float64}` corresponding to the distances in nm from the carbonyl oxygen (index 6) to each of the water oxygens.

We can run a simulation and log these distances.

In [ ]:
sys = System(
    "dipeptide_equil.pdb",
    ff;
    rename_terminal_res=false,
    loggers=(
        distance=GeneralObservableLogger(Owat_distance_wrapper, Vector{Float64}, 10),
        writer=StructureWriter(100, "trajectory.pdb"),
    ),
)
random_velocities!(sys, 300.0u"K")

isfile("trajectory.pdb") && rm("trajectory.pdb")
simulate!(sys, simulator_lang, 10_000) # This will take a little while to run

Now we can use KernelDensity.jl to view the radial distribution function of water molecules around this atom over the simulation.

In [ ]:
distances_flat = vcat(values(sys.loggers.distance)...)
kd = kde(distances_flat; npoints=200)

plot(
    collect(kd.x) .* 10,
    kd.density ./ (kd.x .^ 2), # Off by a normalisation factor
    xlabel="Distance / Å",
    ylabel="Density",
    xlims=(0, 8),
    ylims=(0, 0.5),
)

### Exercise 11

- What does this plot tell you?

---

The double exponential potential is a proposed replacement for the Lennard-Jones potential. It has a physically-motivated exponential repulsion, additional flexibility due to global parameters $\alpha$ and $\beta$, and the energy is finite at zero which is useful for some applications. You can read more about it in [Horton et al. 2023](https://doi.org/10.1039/d3dd00070b). It is defined as:

$$
V(r_{ij}) = \varepsilon_{ij} \left[ \frac{\beta e^\alpha}{\alpha - \beta} \exp \left( -\alpha \frac{r_{ij}}{r_{m,ij}} \right) - \frac{\alpha e^\beta}{\alpha - \beta} \exp \left( -\beta \frac{r_{ij}}{r_{m,ij}} \right) \right]
$$

### Exercise 12

- Define a [custom interaction](https://juliamolsim.github.io/Molly.jl/stable/docs/#Pairwise-interactions) `DoubleExponential` and associated `potential_energy` and `force` functions for the double exponential potential. The struct should take in $\alpha$ and $\beta$ as arguments, for testing you can use `α = 16.766` and `β = 4.427`. You can use `sqrt(atom_i.ϵ * atom_j.ϵ)` for $\varepsilon_{ij}$ and `2^(1/6) * (atom_i.σ + atom_j.σ) / 2` for $r_{m,ij}$. Derive the force function manually, or using Wolfram Alpha, from the relation $F(r_{ij}) = -\frac{\mathrm{d} V(r_{ij})}{\mathrm{d} r_{ij}}$. Also define `Molly.use_neighbors(inter::DoubleExponential) = true`.

Now we can write a function that computes the potential energy for a given interaction and distance.

In [ ]:
function pe(inter, dist)
    a1 = Atom(σ=0.3u"nm", ϵ=0.5u"kJ * mol^-1")
    a2 = Atom(σ=0.3u"nm", ϵ=0.5u"kJ * mol^-1")
    boundary = CubicBoundary(10.0u"nm")
    c1 = SVector(1.0, 1.0, 1.0)u"nm"
    c2 = SVector(dist + 1.0u"nm", 1.0u"nm", 1.0u"nm")
    vec = vector(c1, c2, boundary)
    return potential_energy(inter, vec, c1, c2, a1, a2, boundary)
end

### Exercise 13

- Use the `pe` function to get two arrays of potential energy for the distances `(0.2:0.001:0.6)u"nm"`: one with `LennardJones` and one with `DoubleExponential` where `α = 16.766` and `β = 4.427`. Plot these to compare them.
- Plot variants of the double exponential potential for `(α, β)` in `((10.0, 3.0), (20.0, 3.0), (10.0, 6.0), (20.0, 6.0))`. What effect on the shape of the potential does changing $\alpha$ and $\beta$ have?

We can run a simulation with our custom potential.

In [ ]:
sys = System(
    "dipeptide_equil.pdb",
    ff;
    rename_terminal_res=false,
    loggers=(
        writer=StructureWriter(10, "trajectory.pdb"),
    ),
)
random_velocities!(sys, 300.0u"K")

α, β = 16.766, 4.427
dexp = DoubleExponential(α, β)

sys = System(
    sys;
    # Replace sys.pairwise_inters[1] (the Lennard-Jones interaction) and keep
    #   sys.pairwise_inters[2] (the Coulomb interaction)
    pairwise_inters=(dexp, sys.pairwise_inters[2]),
)

isfile("trajectory.pdb") && rm("trajectory.pdb")
simulate!(sys, simulator_lang, 1000)

### Exercise 14

- Change $\alpha$ and $\beta$ to each of the values in the last exercise, run the simulation and look at the trajectory in PyMOL. What happens in each case? Why?

---

Measurements.jl can be used to propagate uncertainty due to errors. Here we add uncertainty to the system coordinates and velocities and calculate the total energy.

In [ ]:
sys = System(
    "dipeptide_nowater.pdb",
    ff;
    rename_terminal_res=false,
)
random_velocities!(sys, 300.0u"K")
neighbors = find_neighbors(sys)

coords_err = [c .± (abs(randn()) / 100)u"nm"         for c in sys.coords    ]
vels_err   = [v .± (abs(randn()) / 100)u"nm * ps^-1" for v in sys.velocities]
sys = System(sys; coords=coords_err, velocities=vels_err)

total_energy(sys, neighbors)

### Exercise 15

- Which of the kinetic energy and the potential energy contributes more to the uncertainty in the total energy? Why?
- Calculate the temperature of the system and the forces on the atoms. Comment on the level of uncertainty.

### Additional exercises

- How does the Ramachandran plot of alanine dipeptide change with simulation temperature?
- The observed Ramachandran plot will change between short simulations due to poor sampling. How long does it take for this property to converge, such that any simulation of that length gives a similar Ramachandran plot?
- Run a simulation with the `MonteCarloBarostat` to keep the pressure constant at 1 bar. Record the box size over the simulation. How and why does the box size change?
- Remove the Lennard-Jones/double exponential potential from the simulation and see what happens.
- Which of the thermostats available in Molly equilibrates the alanine dipeptide simulation to the desired temperature the fastest? Why?
- Using automatic differentiation with Zygote.jl, show that the force arising from the double exponential potential satisfies $F(r_{ij}) = -\frac{\mathrm{d} V(r_{ij})}{\mathrm{d} r_{ij}}$. How accurate is the automatic differentiation? How does the speed of computing the force this way compare to computing it directly? What are other approaches to calculating the force from the potential energy?